In [1]:
!pip install GitPython
from git import Repo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 181 kB 5.2 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 


In [3]:
HTTPS_REMOTE_URL = 'https://github.com/Bsingstad/Heart-murmur-detection-2022-Simulab.git'
DEST_NAME = 'heart_murmurs'
cloned_repo = Repo.clone_from(HTTPS_REMOTE_URL, DEST_NAME)

In [4]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=56393e3ced13d07e738a64ea383f443ee621c85743095a1b3748c1e9d549cbcb
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json'

kaggle.json
/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [ ]:
#!kaggle datasets download -d bjoernjostein/the-circor-digiscope-phonocardiogram-dataset-v2

 98% 441M/449M [00:07<00:00, 65.8MB/s]
100% 449M/449M [00:07<00:00, 65.3MB/s]


In [6]:
!kaggle datasets download -d bjoernjostein/physionet-challenge-2016

 90% 186M/207M [00:01<00:00, 213MB/s]
100% 207M/207M [00:01<00:00, 162MB/s]


In [7]:
import os, zipfile
try:
  os.mkdir("/content/data/")
except:
  print("data folder allready exists")

In [8]:
import os, zipfile

dir_name = "/content/"
target_dir = "/content/data/"
extension = ".zip"
 
os.chdir(dir_name) # change directory from working dir to dir with files

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(target_dir) # extract file to dir
        zip_ref.close() # close file
        os.remove(file_name) # delete zipped file

In [9]:
os.mknod("/content/heart_murmurs/__init__.py")

In [10]:
def replace_line(file_name, line_num, text):
    lines = open(file_name, 'r').readlines()
    lines[line_num] = text
    out = open(file_name, 'w')
    out.writelines(lines)
    out.close()

In [15]:
replace_line('/content/heart_murmurs/run_model.py',12,"from .helper_code import * \n")
replace_line('/content/heart_murmurs/run_model.py',13,"from .team_code import load_challenge_model, run_challenge_model \n")
replace_line('/content/heart_murmurs/cross_validate.py',14,"from .helper_code import * \n")
replace_line('/content/heart_murmurs/evaluate_model.py',16,"from .helper_code import load_patient_data, get_murmur, get_outcome, load_challenge_outputs, compare_strings \n")
replace_line('/content/heart_murmurs/team_code.py',11,"from .helper_code import * \n")
replace_line('/content/heart_murmurs/train_model.py',11,"from .helper_code import is_integer \n")
replace_line('/content/heart_murmurs/train_model.py',12,"from .team_code import train_challenge_model \n")


In [16]:
%load_ext autoreload
%autoreload
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
import matplotlib.pyplot as plt
from scipy import signal
import numpy as np
import tensorflow as tf

In [24]:
!pip install wfdb
import wfdb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 161 kB 5.2 MB/s 


In [18]:
from heart_murmurs.helper_code import *

# have to add a "." in from of the module imports (in the scrips:train_model, run_model and team_code )
from heart_murmurs.train_model import *
from heart_murmurs.run_model import *
from heart_murmurs.team_code import *
from heart_murmurs.evaluate_model import *
from heart_murmurs.cross_validate import *

In [19]:
try:
  os.mkdir("/content/models/")
except:
  print("model folder allreadu exists")

In [20]:
try:
  os.mkdir("/content/results/")
except:
  print("results folder allreadu exists")

In [40]:
directory="/content/data/"
NEW_FREQUENCY = 100
labels = []
data = []
for subfolder in sorted(os.listdir(directory)):
  if subfolder.startswith("training"):
    for pcgfile in os.listdir(os.path.join(directory,subfolder)):
      if pcgfile.endswith(".hea"):
        psg = wfdb.rdsamp(os.path.join(directory,subfolder,pcgfile.split(".")[0]))
        data.append(signal.resample(psg[0].T[0], int((len(psg[0].T[0])/psg[1]['fs']) * NEW_FREQUENCY)))
        labels.append(psg[1]["comments"][0])
labels = np.asarray(labels)
data_training = np.asarray(data)
ohe_training_label = np.zeros(len(labels))
ohe_training_label[np.where(labels == 'Normal')[0]] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  del sys.path[0]


In [41]:
directory="/content/data/"
NEW_FREQUENCY = 100
labels = []
data = []
for subfolder in sorted(os.listdir(directory)):
  if subfolder.startswith("validation"):
    for pcgfile in os.listdir(os.path.join(directory,subfolder)):
      if pcgfile.endswith(".hea"):
        psg = wfdb.rdsamp(os.path.join(directory,subfolder,pcgfile.split(".")[0]))
        data.append(signal.resample(psg[0].T[0], int((len(psg[0].T[0])/psg[1]['fs']) * NEW_FREQUENCY)))
        labels.append(psg[1]["comments"][0])
labels = np.asarray(labels)
data_validation = np.asarray(data)
ohe_validation_label = np.zeros(len(labels))
ohe_validation_label[np.where(labels == 'Normal')[0]] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  from ipykernel import kernelapp as app


In [44]:
np.unique(ohe_validation_label, return_counts=True)

(array([], dtype=float64), array([], dtype=int64))

In [45]:
np.unique(ohe_training_label, return_counts=True)

(array([0., 1.]), array([ 665, 2575]))

In [50]:
data_training[1].shape

(3573,)

In [58]:
def pad_and_truncated_array(data, max_len = 0):
    if max_len == 0:
      for i in data:
          if len(i) > max_len:
              max_len = len(i)
    else:
      pass
    new_arr = np.zeros((len(data),max_len))
    for j in range(len(data)):
      if len(data[j]) < max_len:
        new_arr[j,:len(data[j])] = data[j]
      elif len(data[j]) > max_len:
        new_arr[j,:] = data[j][:max_len]
    return new_arr

In [53]:
32256//5

6451

In [59]:
data_training_padded = pad_and_truncated_array(data_training, 32256//5)

In [61]:
def _inception_module(input_tensor, stride=1, activation='linear', use_bottleneck=True, kernel_size=40, bottleneck_size=32, nb_filters=32):

    if use_bottleneck and int(input_tensor.shape[-1]) > 1:
        input_inception = tf.keras.layers.Conv1D(filters=bottleneck_size, kernel_size=1,
                                              padding='same', activation=activation, use_bias=False)(input_tensor)
    else:
        input_inception = input_tensor

    # kernel_size_s = [3, 5, 8, 11, 17]
    kernel_size_s = [kernel_size // (2 ** i) for i in range(3)]

    conv_list = []

    for i in range(len(kernel_size_s)):
        conv_list.append(tf.keras.layers.Conv1D(filters=nb_filters, kernel_size=kernel_size_s[i],
                                              strides=stride, padding='same', activation=activation, use_bias=False)(
            input_inception))

    max_pool_1 = tf.keras.layers.MaxPool1D(pool_size=3, strides=stride, padding='same')(input_tensor)

    conv_6 = tf.keras.layers.Conv1D(filters=nb_filters, kernel_size=1,
                                  padding='same', activation=activation, use_bias=False)(max_pool_1)

    conv_list.append(conv_6)

    x = tf.keras.layers.Concatenate(axis=2)(conv_list)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation='relu')(x)
    return x

def _shortcut_layer(input_tensor, out_tensor):
    shortcut_y = tf.keras.layers.Conv1D(filters=int(out_tensor.shape[-1]), kernel_size=1,
                                      padding='same', use_bias=False)(input_tensor)
    shortcut_y = tf.keras.layers.BatchNormalization()(shortcut_y)

    x = tf.keras.layers.Add()([shortcut_y, out_tensor])
    x = tf.keras.layers.Activation('relu')(x)
    return x

def build_model(sig_len,n_features, depth=10, use_residual=True):
    input_layer = tf.keras.layers.Input(shape=(sig_len,n_features))

    x = input_layer
    input_res = input_layer

    for d in range(depth):

        x = _inception_module(x)

        if use_residual and d % 3 == 2:
            x = _shortcut_layer(input_res, x)
            input_res = x

    gap_layer = tf.keras.layers.GlobalAveragePooling1D()(x)

    clinical_output = tf.keras.layers.Dense(1, activation='sigmoid', name="clinical_output")(gap_layer)

    model = tf.keras.models.Model(inputs=input_layer, outputs=clinical_output)
    model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics = [tf.keras.metrics.BinaryAccuracy(),
    tf.keras.metrics.AUC(curve='ROC')])
    
    return model

In [62]:
data_training_padded = np.expand_dims(data_training_padded,2)

In [72]:
ohe_training_label = np.expand_dims(ohe_training_label,1)

In [65]:
training_weights = np.unique(ohe_training_label, return_counts=True)[1][0]/np.unique(ohe_training_label, return_counts=True)[1][1]
training_weights_dictionary = {0: 1.0, 1:training_weights}

In [82]:
def scheduler(epoch, lr):
    if epoch == 5:
        return lr * 0.1
    elif epoch == 10:
        return lr * 0.1
    elif epoch == 15:
        return lr * 0.1
    elif epoch == 20:
        return lr * 0.1
    else:
        return lr

In [83]:
lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=0)

In [79]:
data_training_padded = np.nan_to_num(data_training_padded)

In [84]:
model = build_model(data_training_padded.shape[1],data_training_padded.shape[2])

epochs = 25
batch_size = 20

model.fit(x=data_training_padded, y=ohe_training_label, epochs=epochs, batch_size=batch_size,   
            verbose=1, shuffle = True, validation_split = 0.25,
            class_weight=training_weights_dictionary,
            callbacks=[lr_schedule]
          )

Epoch 1/25
122/122 [==============================] - 71s 540ms/step - loss: 0.2352 - binary_accuracy: 0.6296 - auc_4: 0.7932 - val_loss: 1.2937 - val_binary_accuracy: 0.8951 - val_auc_4: 0.6888 - lr: 0.0010
Epoch 2/25
122/122 [==============================] - 65s 533ms/step - loss: 0.2086 - binary_accuracy: 0.6918 - auc_4: 0.8459 - val_loss: 1.6387 - val_binary_accuracy: 0.2543 - val_auc_4: 0.7998 - lr: 0.0010
Epoch 3/25
122/122 [==============================] - 65s 534ms/step - loss: 0.1949 - binary_accuracy: 0.6852 - auc_4: 0.8562 - val_loss: 0.3845 - val_binary_accuracy: 0.8654 - val_auc_4: 0.7744 - lr: 0.0010
Epoch 4/25
122/122 [==============================] - 65s 535ms/step - loss: 0.1889 - binary_accuracy: 0.6996 - auc_4: 0.8676 - val_loss: 0.3853 - val_binary_accuracy: 0.7938 - val_auc_4: 0.8157 - lr: 0.0010
Epoch 5/25
122/122 [==============================] - 65s 536ms/step - loss: 0.1779 - binary_accuracy: 0.7169 - auc_4: 0.8832 - val_loss: 0.3477 - val_binary_accuracy: 

In [85]:
model = build_model(data_training_padded.shape[1],data_training_padded.shape[2])

epochs = 25
batch_size = 20

model.fit(x=data_training_padded, y=ohe_training_label, epochs=epochs, batch_size=batch_size,   
            verbose=1, shuffle = True, 
            class_weight=training_weights_dictionary,
            callbacks=[lr_schedule])

Epoch 1/25
162/162 [==============================] - 81s 473ms/step - loss: 0.2177 - binary_accuracy: 0.6540 - auc_5: 0.8078 - lr: 0.0010
Epoch 2/25
162/162 [==============================] - 78s 483ms/step - loss: 0.1960 - binary_accuracy: 0.7139 - auc_5: 0.8464 - lr: 0.0010
Epoch 3/25
162/162 [==============================] - 79s 489ms/step - loss: 0.1831 - binary_accuracy: 0.6991 - auc_5: 0.8637 - lr: 0.0010
Epoch 4/25
162/162 [==============================] - 80s 492ms/step - loss: 0.1809 - binary_accuracy: 0.7111 - auc_5: 0.8696 - lr: 0.0010
Epoch 5/25
162/162 [==============================] - 81s 500ms/step - loss: 0.1693 - binary_accuracy: 0.7241 - auc_5: 0.8815 - lr: 0.0010
Epoch 6/25
162/162 [==============================] - 81s 500ms/step - loss: 0.1615 - binary_accuracy: 0.7293 - auc_5: 0.8910 - lr: 1.0000e-04
Epoch 7/25
162/162 [==============================] - 81s 500ms/step - loss: 0.1567 - binary_accuracy: 0.7438 - auc_5: 0.8996 - lr: 1.0000e-04
Epoch 8/25
162/162 

In [86]:
model.save_weights("pretrained_model.h5")